In [2]:
from pathlib import Path

BASE_DIR = Path(".")  # à adapter si besoin
print(BASE_DIR.resolve())

C:\Users\phusson\Desktop\Git_Arterm\index_oeuvres


In [ ]:
import json
from pathlib import Path

BASE_DIR = Path(".")  # à adapter si besoin
FOLDERS = {
    "peinture": BASE_DIR / "data" / "entry_peinture",
    "architecture": BASE_DIR / "data" / "entry_architecture"
}

def process_peinture(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    holding = data.pop("holding_institution", None)

    new_data = {}

    for key, value in data.items():
        new_data[key] = value

        # insertion juste après dateCreated
        if key == "dateCreated" and holding:
            new_data["location"] = {
                "type": "holding_institution",
                "institution": {
                    "name": holding.get("name", ""),
                    "place": holding.get("place", ""),
                    "inventory_number": holding.get("inventory_number", ""),
                    "url": holding.get("URL", "")
                }
            }

    return new_data



def process_architecture(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    loc = data.get("location")

    # on évite de retraiter des fichiers déjà migrés
    if loc and "type" not in loc:
        data["location"] = {
            "type": "place",
            "place": {
                "city": loc.get("city", ""),
                "country": loc.get("country", ""),
                "coordinates": loc.get("coordinates", {
                    "latitude": "",
                    "longitude": ""
                })
            }
        }

    return data


def process_folder(folder, processor):
    for json_file in folder.glob("*.json"):
        new_data = processor(json_file)

        with open(json_file, "w", encoding="utf-8") as f:
            json.dump(new_data, f, ensure_ascii=False, indent=2)

        print(f"✔ {json_file.name} mis à jour")


if __name__ == "__main__":
    process_folder(FOLDERS["peinture"], process_peinture)
    process_folder(FOLDERS["architecture"], process_architecture)


✔ AndreaAndreaniTriompheGrave.json mis à jour
✔ AndreaDelCastagnoExecutionConjures.json mis à jour
✔ AnonymeHelene.json mis à jour
✔ ApelleCalomnie.json mis à jour
✔ ApelleCampaspe.json mis à jour
✔ ArcheAlliance.json mis à jour
✔ BotticelliDivineComedie.json mis à jour
✔ CarracheGalerieFarnese.json mis à jour
✔ CavalliniCrucifixSaintPaul.json mis à jour
✔ CopieLeonardCeneParis.json mis à jour
✔ CorregeCoupoleAssomption.json mis à jour
✔ DaCarpiVeroniquePaulPierre.json mis à jour
✔ DelSartoLeonX.json mis à jour
✔ DurerPetitePassion.json mis à jour
✔ FilippinoLippiChapelleCarafa.json mis à jour
✔ FilippoLippiViergeCeinturePrato.json mis à jour
✔ FraAngelicoChapelleNicoline.json mis à jour
✔ GiottoNacelleSaintPierreRome.json mis à jour
✔ LeonardAdamEve.json mis à jour
✔ LeonardAdorationMages.json mis à jour
✔ LeonardAnghiari.json mis à jour
✔ LeonardAssomptionMilan.json mis à jour
✔ LeonardCartonBurlington.json mis à jour
✔ LeonardCene.json mis à jour
✔ LeonardCeneMilan.json mis à jour
✔

In [4]:
import json
from pathlib import Path

PEINTURE_DIR = BASE_DIR / "data" / "entry_peinture"

def reorder_location_after_datecreated(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # on vérifie que location existe et est du bon type
    location = data.get("location")
    if not location or location.get("type") != "holding_institution":
        return False

    new_data = {}

    for key, value in data.items():
        if key == "location":
            continue  # on la réinsère plus loin

        new_data[key] = value

        if key == "dateCreated":
            new_data["location"] = location

    # sécurité : si dateCreated n'existait pas
    if "location" not in new_data:
        new_data["location"] = location

    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(new_data, f, ensure_ascii=False, indent=2)

    return True


count = 0
for json_file in PEINTURE_DIR.glob("*.json"):
    if reorder_location_after_datecreated(json_file):
        count += 1

print(f"✔ {count} notices peinture corrigées (ordre de location)")


✔ 136 notices peinture corrigées (ordre de location)
